# Evaluation of Training for Model Selection (Using Dev Set)

In [ ]:
!pip install torch
!pip install torchmetrics
from torch.nn import functional as F
import torch
!pip install transformers
from transformers import RobertaForSequenceClassification, Trainer
from google.colab import drive
!pip3 install pickle5
import pickle5 as pickle
import os
## needed for this task in particular
from torch.utils.data import DataLoader
from torchmetrics import RetrievalMAP

In [ ]:
def save_as_pickle(obj, filename):
    """
    save an object in a pickle file dump
    :param obj: object to dump
    :param filename: target file
    :return:
    """
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with open(filename, 'wb') as file:
        pickle.dump(obj, file, protocol=pickle.HIGHEST_PROTOCOL)


def load_pickle(filename):
    """
    load an object from a given pickle file
    :param filename: source file
    :return: loaded object
    """
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [ ]:
drive.mount('/content/drive', force_remount=True)  # use force_remount=True param after upload of new data

Mounted at /content/drive


In [ ]:
# file and folder names
ir_project_drive_folder = "Master/2 - FSS 2021/Information Retrieval/IR Projekt"  # TODO: geht das für alle?
#ir_project_drive_folder = "IR Projekt"  # TODO: geht das für alle?
full_ir_project_drive_folder = "/content/drive/My Drive/{}/".format(ir_project_drive_folder)

## encoding 
bm25_retrieved_dev_pl_map_filename = full_ir_project_drive_folder + "data/wikipedia/neural-retrieval-datasets/dev_pl_map.pkl"

nr_models_folder = full_ir_project_drive_folder + 'saved_models/neural_retrieval'
# first cycle (random irrelevant) - equal
model_path_roberta_first_10000_10000_wd_2e1 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-250'
model_path_roberta_first_10000_10000_wd_2e2 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-500'
model_path_roberta_first_10000_10000_wd_2e3 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-750'
model_path_roberta_first_10000_10000_wd_2e4 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-1000'
model_path_roberta_first_10000_10000_wd_2e5 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-1250'
model_path_roberta_first_10000_10000_wd_2e6 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-1500'
model_path_roberta_first_10000_10000_wd_2e7 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-1750'
model_path_roberta_first_10000_10000_wd_2e8 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-2000'
model_path_roberta_first_10000_10000_wd_2e9 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-2250'
model_path_roberta_first_10000_10000_wd_2e10 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-2500'
model_path_roberta_first_10000_10000_wd_2e11 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-2750'
model_path_roberta_first_10000_10000_wd_2e12 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-3000'
model_path_roberta_first_10000_10000_wd_2e13 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-3250'
model_path_roberta_first_10000_10000_wd_2e14 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-3500'
model_path_roberta_first_10000_10000_wd_2e15 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-3750'
model_path_roberta_first_10000_10000_wd_2e16 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-4000'
model_path_roberta_first_10000_10000_wd_2e17 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-4250'
model_path_roberta_first_10000_10000_wd_2e18 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-4500'
model_path_roberta_first_10000_10000_wd_2e19 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-4750'
model_path_roberta_first_10000_10000_wd_2e20 = nr_models_folder + '/roberta_first_10000_10000_wd_2e/checkpoint-5000'

# first cycle (random irrelevant) - approximate
model_path_roberta_first_5000_15000_wd_2e1 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-250'
model_path_roberta_first_5000_15000_wd_2e2 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-500'
model_path_roberta_first_5000_15000_wd_2e3 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-750'
model_path_roberta_first_5000_15000_wd_2e4 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-1000'
model_path_roberta_first_5000_15000_wd_2e5 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-1250'
model_path_roberta_first_5000_15000_wd_2e6 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-1500'
model_path_roberta_first_5000_15000_wd_2e7 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-1750'
model_path_roberta_first_5000_15000_wd_2e8 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-2000'
model_path_roberta_first_5000_15000_wd_2e9 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-2250'
model_path_roberta_first_5000_15000_wd_2e10 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-2500'
model_path_roberta_first_5000_15000_wd_2e11 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-2750'
model_path_roberta_first_5000_15000_wd_2e12 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-3000'
model_path_roberta_first_5000_15000_wd_2e13 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-3250'
model_path_roberta_first_5000_15000_wd_2e14 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-3500'
model_path_roberta_first_5000_15000_wd_2e15 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-3750'
model_path_roberta_first_5000_15000_wd_2e16 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-4000'
model_path_roberta_first_5000_15000_wd_2e17 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-4250'
model_path_roberta_first_5000_15000_wd_2e18 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-4500'
model_path_roberta_first_5000_15000_wd_2e19 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-4750'
model_path_roberta_first_5000_15000_wd_2e20 = nr_models_folder + '/roberta_first_5000_15000_wd_2e/checkpoint-5000'

# first cycle (random irrelevant) - real
model_path_roberta_first_720_19280_wd_2e1 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-250'
model_path_roberta_first_720_19280_wd_2e2 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-500'
model_path_roberta_first_720_19280_wd_2e3 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-750'
model_path_roberta_first_720_19280_wd_2e4 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-1000'
model_path_roberta_first_720_19280_wd_2e5 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-1250'
model_path_roberta_first_720_19280_wd_2e6 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-1500'
model_path_roberta_first_720_19280_wd_2e7 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-1750'
model_path_roberta_first_720_19280_wd_2e8 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-2000'
model_path_roberta_first_720_19280_wd_2e9 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-2250'
model_path_roberta_first_720_19280_wd_2e10 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-2500'
model_path_roberta_first_720_19280_wd_2e11 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-2750'
model_path_roberta_first_720_19280_wd_2e12 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-3000'
model_path_roberta_first_720_19280_wd_2e13 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-3250'
model_path_roberta_first_720_19280_wd_2e14 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-3500'
model_path_roberta_first_720_19280_wd_2e15 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-3750'
model_path_roberta_first_720_19280_wd_2e16 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-4000'
model_path_roberta_first_720_19280_wd_2e17 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-4250'
model_path_roberta_first_720_19280_wd_2e18 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-4500'
model_path_roberta_first_720_19280_wd_2e19 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-4750'
model_path_roberta_first_720_19280_wd_2e20 = nr_models_folder + '/roberta_first_720_19280_wd_2e/checkpoint-5000'

# second cycle (random irrelevant) - 5000 - 15000
model_path_roberta_second_5000_15000_wd_2e1 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-250'
model_path_roberta_second_5000_15000_wd_2e2 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-500'
model_path_roberta_second_5000_15000_wd_2e3 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-750'
model_path_roberta_second_5000_15000_wd_2e4 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-1000'
model_path_roberta_second_5000_15000_wd_2e5 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-1250'
model_path_roberta_second_5000_15000_wd_2e6 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-1500'
model_path_roberta_second_5000_15000_wd_2e7 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-1750'
model_path_roberta_second_5000_15000_wd_2e8 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-2000'
model_path_roberta_second_5000_15000_wd_2e9 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-2250'
model_path_roberta_second_5000_15000_wd_2e10 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-2500'
model_path_roberta_second_5000_15000_wd_2e11 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-2750'
model_path_roberta_second_5000_15000_wd_2e12 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-3000'
model_path_roberta_second_5000_15000_wd_2e13 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-3250'
model_path_roberta_second_5000_15000_wd_2e14 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-3500'
model_path_roberta_second_5000_15000_wd_2e15 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-3750'
model_path_roberta_second_5000_15000_wd_2e16 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-4000'
model_path_roberta_second_5000_15000_wd_2e17 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-4250'
model_path_roberta_second_5000_15000_wd_2e18 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-4500'
model_path_roberta_second_5000_15000_wd_2e19 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-4750'
model_path_roberta_second_5000_15000_wd_2e20 = nr_models_folder + '/roberta_second_5000_15000_wd_2e/checkpoint-5000'

# second cycle (random hard) - 5000 - 15000
model_path_roberta_second_randhard_5000_15000_wd_2e1 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-250'
model_path_roberta_second_randhard_5000_15000_wd_2e2 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-500'
model_path_roberta_second_randhard_5000_15000_wd_2e3 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-750'
model_path_roberta_second_randhard_5000_15000_wd_2e4 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-1000'
model_path_roberta_second_randhard_5000_15000_wd_2e5 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-1250'
model_path_roberta_second_randhard_5000_15000_wd_2e6 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-1500'
model_path_roberta_second_randhard_5000_15000_wd_2e7 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-1750'
model_path_roberta_second_randhard_5000_15000_wd_2e8 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-2000'
model_path_roberta_second_randhard_5000_15000_wd_2e9 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-2250'
model_path_roberta_second_randhard_5000_15000_wd_2e10 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-2500'
model_path_roberta_second_randhard_5000_15000_wd_2e11 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-2750'
model_path_roberta_second_randhard_5000_15000_wd_2e12 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-3000'
model_path_roberta_second_randhard_5000_15000_wd_2e13 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-3250'
model_path_roberta_second_randhard_5000_15000_wd_2e14 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-3500'
model_path_roberta_second_randhard_5000_15000_wd_2e15 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-3750'
model_path_roberta_second_randhard_5000_15000_wd_2e16 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-4000'
model_path_roberta_second_randhard_5000_15000_wd_2e17 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-4250'
model_path_roberta_second_randhard_5000_15000_wd_2e18 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-4500'
model_path_roberta_second_randhard_5000_15000_wd_2e19 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-4750'
model_path_roberta_second_randhard_5000_15000_wd_2e20 = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e/checkpoint-5000'

# second cycle (max hard) - 5000 - 15000
model_path_roberta_second_maxhard_5000_15000_wd_2e1 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-250'
model_path_roberta_second_maxhard_5000_15000_wd_2e2 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-500'
model_path_roberta_second_maxhard_5000_15000_wd_2e3 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-750'
model_path_roberta_second_maxhard_5000_15000_wd_2e4 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-1000'
model_path_roberta_second_maxhard_5000_15000_wd_2e5 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-1250'
model_path_roberta_second_maxhard_5000_15000_wd_2e6 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-1500'
model_path_roberta_second_maxhard_5000_15000_wd_2e7 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-1750'
model_path_roberta_second_maxhard_5000_15000_wd_2e8 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-2000'
model_path_roberta_second_maxhard_5000_15000_wd_2e9 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-2250'
model_path_roberta_second_maxhard_5000_15000_wd_2e10 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-2500'
model_path_roberta_second_maxhard_5000_15000_wd_2e11 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-2750'
model_path_roberta_second_maxhard_5000_15000_wd_2e12 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-3000'
model_path_roberta_second_maxhard_5000_15000_wd_2e13 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-3250'
model_path_roberta_second_maxhard_5000_15000_wd_2e14 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-3500'
model_path_roberta_second_maxhard_5000_15000_wd_2e15 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-3750'
model_path_roberta_second_maxhard_5000_15000_wd_2e16 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-4000'
model_path_roberta_second_maxhard_5000_15000_wd_2e17 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-4250'
model_path_roberta_second_maxhard_5000_15000_wd_2e18 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-4500'
model_path_roberta_second_maxhard_5000_15000_wd_2e19 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-4750'
model_path_roberta_second_maxhard_5000_15000_wd_2e20 = nr_models_folder + '/roberta_second_maxhard_5000_15000_wd_2e/checkpoint-5000'

## Load Data


In [ ]:
encodings= load_pickle(RoBERTa_encoding_path_100)
input = load_pickle(bm25_retrieved_dev_pl_map_filename)
targets = input[1]
indexes = input[0]
encodings = input[2]

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, boundaries=None):
    if boundaries is not None:
      self.encodings = {key: val[boundaries[0]: boundaries[1]] for key, val in encodings.items()}
    else:
      self.encodings = encodings

  def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      return item

  def __len__(self):
      return len(self.encodings['input_ids'])


def get_preds(dataset):
  prediction_output = trainer.predict(dataset)
  predictions = prediction_output.predictions
  preds = torch.tensor(predictions)
  res = F.softmax(preds, dim = 1)
  results = res[:,1]
  del predictions
  del prediction_output
  del preds
  del res
  torch.cuda.empty_cache()
  return results

sub_dataset = Dataset(encodings)

# Re-Ranking (first)


## 10000 - 10000

### Checkpoint 1

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e1)
trainer = Trainer(
    model = model
)
pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3495)

### Checkpoint 2

In [ ]:
# del model
# del pred_array
# del map
# del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e2)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.2085)

### Checkpoint 3

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e3)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3199)

### Checkpoint 4

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e4)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3262)

### Checkpoint 5

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e5)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3523)

### Checkpoint 6

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e6)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.2300)

### Checkpoint 7

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e7)

trainer = Trainer(
    model = model
)


pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.2258)

### Checkpoint 8

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e8)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3156)

### Checkpoint 9

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e9)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4546)

### Checkpoint 10

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e10)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3332)

### Checkpoint 11

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e11)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4651)

### Checkpoint 12

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e12)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3332)

### Checkpoint 13

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e13)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3273)

### Checkpoint 14

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e14)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3508)

### Checkpoint 15

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e15)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4033)

### Checkpoint 16

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e16)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3544)

### Checkpoint 17

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e17)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3633)

### Checkpoint 18

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e18)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4619)

### Checkpoint 19

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e19)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4146)

### Checkpoint 20

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_10000_10000_wd_2e20)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4771)

## 5000 - 15000 Split

### Checkpoint 1

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e1)
trainer = Trainer(
    model = model
)
pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 2

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e2)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 3

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e3)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 4

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e4)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 5

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e5)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 6

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e6)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 7

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e7)

trainer = Trainer(
    model = model
)


pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 8

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e8)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 9

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e9)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 10

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e10)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 11

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e11)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 12

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e12)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 13

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e13)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 14

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e14)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 15

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e15)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 16

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e16)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.5581)

### Checkpoint 17

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e17)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.7449)

### Checkpoint 18

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e18)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.2093)

### Checkpoint 19

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e19)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.7032)

### Checkpoint 20

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e20)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.7010)

## 720 - 19280 Split

### Checkpoint 1

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e1)
trainer = Trainer(
    model = model
)
pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0614)

### Checkpoint 2

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e2)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0799)

### Checkpoint 3

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e3)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.1051)

### Checkpoint 4

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e4)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0638)

### Checkpoint 5

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e5)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0663)

### Checkpoint 6

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e6)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0667)

### Checkpoint 7

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e7)

trainer = Trainer(
    model = model
)


pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0557)

### Checkpoint 8

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e8)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 9

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e9)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 10

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e10)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 11

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e11)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 12

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e12)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 13

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e13)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 14

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e14)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 15

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e15)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 16

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e16)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 17

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e17)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 18

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e18)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 19

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e19)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 20

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_720_19280_wd_2e20)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


KeyboardInterrupt: ignored

In [ ]:
#encodings= load_pickle(RoBERTa_encoding_path_100)
input = load_pickle(bm25_retrieved_dev_pl_map_filename)
targets = input[1]
indexes = input[0]
encodings = input[2]
print(targets)
print(indexes)
print(encodings)

tensor([ True,  True, False,  ..., False, False, False])
tensor([ 0,  0,  0,  ..., 99, 99, 99])
{'input_ids': tensor([[    0, 12375,    21,  ...,    23, 22226,     2],
        [    0, 12375,    21,  ...,   380,  2441,     2],
        [    0, 12375,    21,  ...,   380,  2441,     2],
        ...,
        [    0,  2264,    21,  ...,  2176,  3225,     2],
        [    0,  2264,    21,  ...,   149, 11775,     2],
        [    0,  2264,    21,  ...,    64,   293,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, boundaries=None):
    if boundaries is not None:
      self.encodings = {key: val[boundaries[0]: boundaries[1]] for key, val in encodings.items()}
    else:
      self.encodings = encodings

  def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      return item

  def __len__(self):
      return len(self.encodings['input_ids'])


def get_preds(dataset):
  prediction_output = trainer.predict(dataset)
  predictions = prediction_output.predictions
  preds = torch.tensor(predictions)
  res = F.softmax(preds, dim = 1)
  results = res[:,1]
  del predictions
  del prediction_output
  del preds
  del res
  torch.cuda.empty_cache()
  return results

sub_dataset = Dataset(encodings)

# Re-Ranking (second)

## 5000 - 15000 Split - random irrelevant

### Checkpoint 1

In [ ]:
# del model
# del pred_array
# del map
# del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e1)
trainer = Trainer(
    model = model
)
pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3552)

### Checkpoint 2

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e2)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3833)

### Checkpoint 3

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e3)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4172)

### Checkpoint 4

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e4)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4076)

### Checkpoint 5

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e5)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.7277)

### Checkpoint 6

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e6)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.7383)

### Checkpoint 7

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e7)

trainer = Trainer(
    model = model
)


pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.3725)

### Checkpoint 8

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e8)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4732)

### Checkpoint 9

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e9)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.5084)

### Checkpoint 10

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e10)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.6423)

### Checkpoint 11

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e11)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.5512)

### Checkpoint 12

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e12)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.5247)

### Checkpoint 13

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e13)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.6348)

### Checkpoint 14

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e14)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4629)

### Checkpoint 15

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e15)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.6814)

### Checkpoint 16

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e16)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.5892)

### Checkpoint 17

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e17)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.5937)

### Checkpoint 18

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e18)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.4705)

### Checkpoint 19

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e19)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.5555)

### Checkpoint 20

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_5000_15000_wd_2e20)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.5153)

## 5000 - 15000 Split - random hard

### Checkpoint 1

In [ ]:
# del model
# del pred_array
# del map
# del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e1)
trainer = Trainer(
    model = model
)
pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.7878)

### Checkpoint 2

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e2)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.7920)

### Checkpoint 3

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e3)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.7920)

### Checkpoint 4

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e4)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8061)

### Checkpoint 5

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e5)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8040)

### Checkpoint 6

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e6)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8017)

### Checkpoint 7

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e7)

trainer = Trainer(
    model = model
)


pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8004)

### Checkpoint 8

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e8)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8122)

### Checkpoint 9

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e9)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8189)

### Checkpoint 10

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e10)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8156)

### Checkpoint 11

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e11)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8122)

### Checkpoint 12

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e12)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8139)

### Checkpoint 13

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e13)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8171)

### Checkpoint 14

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e14)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8174)

### Checkpoint 15

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e15)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8231)

### Checkpoint 16

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e16)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8117)

### Checkpoint 17

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e17)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8179)

### Checkpoint 18

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e18)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8164)

### Checkpoint 19

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e19)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8161)

### Checkpoint 20

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e20)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.8204)

## 5000 - 15000 Split - max hard

### Checkpoint 1

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e1)
trainer = Trainer(
    model = model
)
pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0614)

### Checkpoint 2

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e2)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0799)

### Checkpoint 3

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e3)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.1051)

### Checkpoint 4

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e4)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0638)

### Checkpoint 5

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e5)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0663)

### Checkpoint 6

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e6)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0667)

### Checkpoint 7

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e7)

trainer = Trainer(
    model = model
)


pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


tensor(0.0557)

### Checkpoint 8

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e8)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 9

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e9)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 10

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e10)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 11

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e11)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 12

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e12)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 13

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e13)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 14

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e14)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 15

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e15)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 16

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e16)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 17

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e17)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


### Checkpoint 18

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e18)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 19

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e19)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

### Checkpoint 20

In [ ]:
del model
del pred_array
del map
del trainer

model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_maxhard_5000_15000_wd_2e20)

trainer = Trainer(
    model = model
)

pred_array = get_preds(sub_dataset)
pred_array
map = RetrievalMAP()
map(pred_array, targets, indexes=indexes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


KeyboardInterrupt: ignored